# Python Web Scraping Tutorial using BeutifulSoup

- Goal: web scrape the weather forecasts from the National Weather Service using Python

# Part 1
- First, let's download some sample content from http://dataquestio.github.io/web-scraping-pages/simple.html using the requests library

In [1]:
import requests

# note: this returns a Response object, where the number indicates success/error
# generally, 2__ => success, 4__ or 5__ => error
page = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

In [2]:
# print out HTML content of page using .content
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

# Part 2: Parse a page with BeautifulSoup

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
# print out HTML content in a nice format
# note: this makes the tags nested
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [8]:
# get all elements at the top level of the page
list(soup.children)

['html', '\n', <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

- above output: tells us there are two tags at the top level => "<!DOCTYPE html>" and html tag as well as a newline character
- let's output the type for each element in the list

In [9]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

- above output: all BeautifulSoup objects. 
1. Doctype => contains infor about the type of the document.
2. NavigableString => represents the text found in the HTML document.
3. (most common) Tag => contains other nested tags and allows us to navigate through an HTML document to extract other tags and text.

In [14]:
# take third item in list to get html tag and its children
html = list(soup.children)[2]
html

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<p>Here is some simple content for this page.</p>
</body>
</html>

In [15]:
# call children method on html
list(html.children)

['\n', <head>
 <title>A simple example page</title>
 </head>, '\n', <body>
 <p>Here is some simple content for this page.</p>
 </body>, '\n']

In [19]:
# to extract text inside <p> tag, we dive into the body
body = list(html.children)[3]
body

<body>
<p>Here is some simple content for this page.</p>
</body>

In [20]:
# get the p tag by finding the children of the body tag
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [26]:
# isolate p tag
p = list(body.children)[1]

# get text inside the tag
p.get_text()

'Here is some simple content for this page.'

# Find all instances of a tag at once

- the above is nice if we want to naviagte a page, but let's find all the instances of a tag on a page

In [27]:
soup = BeautifulSoup(page.content, 'html.parser')

# note: find_all returns a list
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [28]:
# since a list is returned, we need to loop through/use list indexing, to extract text
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

In [29]:
# if you want to find the first instance of a tag, use find() method
soup.find('p')

<p>Here is some simple content for this page.</p>

# Searching for tags by class and id

- Note: classes and id's are used by CSS to figure out which HTML elements to apply certain styles to.
- We can also use them when scraping to specify elements we want to scrape
- Use http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html as an example

In [30]:
page = requests.get("http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [31]:
# Use find_all to search for items by class or id
# e.g. find p tags that have the class "outer-text"
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [32]:
# e.g. 2 find tags with id = "first"
soup.find_all(id = "first")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

# Using CSS Selectors

- We can search for items using CSS selectors.
- Note: CSS selectors are how the CSS language allows developers to specify HTML tags to style.
- Examples:
    1. p a = finds all a tags inside of a p tag.
    2. body p a = finds all a tags inside of a p tag inside of a body tag.
    3. html body = finds all body tags inside of an html tag.
    4. p.outer-text = finds all p tags with a class of outer_text.
    5. p#first = finds all p tags with an id of first.
    6. body p.outer-text = finds all p tags with a class of outer-text inside of a body tag.
- to find such tags, use the select() method

In [33]:
soup.select("div p")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>, <p class="inner-text">
                 Second paragraph.
             </p>]

# Download Weather Data

- Let's download weather data from Vernon Hills, IL.
- url: https://forecast.weather.gov/MapClick.php?lon=-87.95749379978636&lat=42.22480081460347#.WphYqJPwaAw
- Goal: extract data about the extended forecast
- Explore/inspect the page using the Chrom Devtools (View->Developer->Developr Tools).
- Find the tag that represents the data we want to extract (this ends up being that the extended forecasts are contained in a div with id = "seven-day-forecast" and the specific elements have class="forecast-tombstone"). We now know wnough to download the page and start parsing it. So we do the following.
- Steps:
    1. Download web page containing the forecast.
    2. Create a BeautifulSoup class to parse the page.
    3. Find the div with id = seven-day-forecast, and assign to seven_day variable.
    4. Inside seven_day, find each individual forecast tiem.
    5. Extract and print the first forecast item.

In [35]:
# Download web page
page = requests.get("https://forecast.weather.gov/MapClick.php?lon=-87.95749379978636&lat=42.22480081460347#.WphZZpPwY3i")

In [36]:
# Create soup object
soup = BeautifulSoup(page.content, 'html.parser')

In [38]:
# find id = "seven-day-forecast" 
seven_day = soup.find(id = "seven-day-forecast")

In [50]:
# find elements with class = "tombstone-container"
# these are our 7 day forecast items
forecast_items = seven_day.find_all(class_= "tombstone-container")

In [51]:
# get this afternoon's temperature (i.e. the first mentioned forecast)
thisAfternoon = forecast_items[0]
print(thisAfternoon.prettify())

<div class="tombstone-container">
 <p class="period-name">
  This
  <br/>
  Afternoon
 </p>
 <p>
  <img alt="This Afternoon: A 20 percent chance of rain after 5pm.  Mostly cloudy, with a high near 41. North wind 10 to 15 mph, with gusts as high as 25 mph. " class="forecast-icon" src="newimages/medium/ra20.png" title="This Afternoon: A 20 percent chance of rain after 5pm.  Mostly cloudy, with a high near 41. North wind 10 to 15 mph, with gusts as high as 25 mph. "/>
 </p>
 <p class="short-desc">
  Slight Chance
  <br/>
  Rain
 </p>
 <p class="temp temp-high">
  High: 41 °F
 </p>
</div>


- Note in the above, we have 4 things we would like to extract:
    1. Name of forecast item ("this afternoon")
    2. Description of conditions
    3. Short description of conditions
    4. Temperature high

In [52]:
# First extract forecast item, short description, and temperature first
# do this because theyre similar
period = thisAfternoon.find(class_="period-name").get_text()
shortDescription = thisAfternoon.find(class_="short-desc").get_text()
temp = thisAfternoon.find(class_="temp").get_text()

print(period)
print(shortDescription)
print(temp)

ThisAfternoon
Slight ChanceRain
High: 41 °F


In [53]:
# extract title attribute from img tag
img = thisAfternoon.find("img")
longDescription = img['title']
print(longDescription)

This Afternoon: A 20 percent chance of rain after 5pm.  Mostly cloudy, with a high near 41. North wind 10 to 15 mph, with gusts as high as 25 mph. 


# Extract all the information from the page

- Now do everything at once:
    1. Select all items with the class = "period-name" inside an item with the class tombstone-container in seven_day variable (i.e. css selector)
    2. Use a list comprehension to call the get_text method on each BeautifulSoup object

In [56]:
# get all periods
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['ThisAfternoon',
 'Tonight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday']

In [57]:
# get all short descriptions
shortDescription_tags = seven_day.select(".tombstone-container .short-desc")
shortDescriptions = [sd.get_text() for sd in shortDescription_tags]
shortDescriptions

['Slight ChanceRain',
 'Partly Cloudy',
 'Sunny',
 'Mostly Clear',
 'Sunny',
 'Mostly Clear',
 'Sunny',
 'Mostly Cloudy',
 'Rain']

In [58]:
# get all temperatures
temp_tags = seven_day.select(".tombstone-container .temp")
temperatures = [t.get_text() for t in temp_tags]
temperatures

['High: 41 °F',
 'Low: 27 °F',
 'High: 43 °F',
 'Low: 26 °F',
 'High: 44 °F',
 'Low: 29 °F',
 'High: 49 °F',
 'Low: 37 °F',
 'High: 47 °F']

In [64]:
# get all long descriptions
longDescription_tags = seven_day.select(".tombstone-container img")
longDescriptions = [ld["title"] for ld in longDescription_tags]
longDescriptions

['This Afternoon: A 20 percent chance of rain after 5pm.  Mostly cloudy, with a high near 41. North wind 10 to 15 mph, with gusts as high as 25 mph. ',
 'Tonight: Partly cloudy, with a low around 27. North northwest wind around 10 mph. ',
 'Friday: Sunny, with a high near 43. North northwest wind 5 to 10 mph becoming east northeast in the afternoon. ',
 'Friday Night: Mostly clear, with a low around 26. East northeast wind around 5 mph. ',
 'Saturday: Sunny, with a high near 44. East southeast wind 5 to 10 mph. ',
 'Saturday Night: Mostly clear, with a low around 29.',
 'Sunday: Sunny, with a high near 49.',
 'Sunday Night: Mostly cloudy, with a low around 37.',
 'Monday: Rain, mainly after noon.  Mostly cloudy, with a high near 47.']

# Combine data into a pandas df

- let's combine our data into a pandas df and analyze it.
- note: a df is an object that can store tabular data.

In [65]:
import pandas as pd

weather = pd.DataFrame({
    "period": periods,
    "shortDescription": shortDescriptions,
    "temp": temperatures,
    "longDescription": longDescriptions
})

weather

,longDescription,period,shortDescription,temp
0,This Afternoon: A 20 percent chance of rain af...,ThisAfternoon,Slight ChanceRain,High: 41 °F
1,"Tonight: Partly cloudy, with a low around 27. ...",Tonight,Partly Cloudy,Low: 27 °F
2,"Friday: Sunny, with a high near 43. North nort...",Friday,Sunny,High: 43 °F
3,"Friday Night: Mostly clear, with a low around ...",FridayNight,Mostly Clear,Low: 26 °F
4,"Saturday: Sunny, with a high near 44. East sou...",Saturday,Sunny,High: 44 °F
5,"Saturday Night: Mostly clear, with a low aroun...",SaturdayNight,Mostly Clear,Low: 29 °F
6,"Sunday: Sunny, with a high near 49.",Sunday,Sunny,High: 49 °F
7,"Sunday Night: Mostly cloudy, with a low around...",SundayNight,Mostly Cloudy,Low: 37 °F
8,"Monday: Rain, mainly after noon. Mostly cloud...",Monday,Rain,High: 47 °F
